In [62]:
import pandas as pd
from web3 import Web3
from web3.middleware import geth_poa_middleware

In [63]:
chain = 'eth'
query_mode = 'http'
w3 = None
data = {
    'eth': {
        'rpc': '/eth/eth_node/node/geth.ipc',
        'http': 'https://mainnet.infura.io/v3/e0a4e987f3ff4f4fa9aa21bb08f09ef5',
        'datafile': 'data/one_day_exit_scam_eth.csv',
        'edgefile': 'data/new_tx_eth.csv',
        'nodefile': 'data/new_address_eth.csv'
    },
    'bsc': {
        'rpc': '',
        'http': 'https://bsc-dataseed.binance.org/',
        'datafile': 'one_day_exit_scam_bsc.csv',
        'edgefile': 'data/new_tx_bsc.csv',
        'nodefile': 'data/new_address_bsc.csv'
    }
}

In [64]:
def get_w3():
    tmp = data[chain][query_mode]
    w3_data = {
        'eth': {
            'rpc': Web3(Web3.IPCProvider(tmp)),
            'http': Web3(Web3.HTTPProvider(tmp))
        },
        'bsc': {
            'rpc': None,
            'http': Web3(Web3.HTTPProvider(tmp)).middleware_onion.inject(geth_poa_middleware, layer=0)
        } 
    }
    return w3_data[chain][query_mode]

In [65]:
def preprocessing():
    datafile = data[chain]['datafile']
    df = pd.read_csv(datafile)
    address_df = df.\
        sort_values(["block_number_remove"]).\
        drop_duplicates(subset=["from"], keep="last").\
        reset_index(drop=True)\
        [["from", "block_number_remove"]].\
        rename({"from":"address", "block_number_remove":"use_untill"}, axis="columns")
    address_df["level"] = 0
    return address_df

In [66]:
import requests

url = "https://eth-mainnet.g.alchemy.com/v2/demo"

payload = {
    "id": 1,
    "jsonrpc": "2.0",
    "method": "alchemy_getAssetTransfers",
    "params": [
        {
            "fromBlock": "0x0",
            "toBlock": "latest",
            "category": ["external"],
            "withMetadata": False,
            "excludeZeroValue": True,
            "maxCount": "0x3e8",
            "toAddress": ""
        }
    ]
}
headers = {
    "accept": "application/json",
    "content-type": "application/json"
}

In [67]:
w3 = get_w3()
address_df = preprocessing()
address_df.to_csv(data[chain]['nodefile'], index=False)
address_df.to_csv('data/tocompute.csv', index=False)
with open(data[chain]['edgefile'], "w", encoding="UTF8") as tx_file:
    tx_file.write("from,to,value,effectiveGasPrice,gasUsed,hash,input,blockNumber,level,status\n")

In [60]:
def multi():
    address_df = set(preprocessing()['address'].values)
    curr_level_address_df = pd.read_csv('data/tocompute.csv')
    new_level = 1
    #generate new level of edges and nodes
    for index, row in curr_level_address_df.iterrows():
        payload['params'][0]['toAddress'] = row["address"]
        payload['params'][0]['toBlock'] = hex(row["use_untill"])
        try:
            response = requests.post(url, json=payload, headers=headers)
            history_txs = response.json()['result']['transfers']
            txlist = []
            adlist = []
            for tx in history_txs:
                tx = {**w3.eth.get_transaction(tx['hash']), **w3.eth.get_transaction_receipt(tx['hash'])}
                txlist.append("{},{},{},{},{},{},{},{},{},{}\n".format(
                    tx["from"], tx["to"], w3.fromWei(tx["value"], 'ether'), tx["effectiveGasPrice"], tx["gasUsed"], tx["hash"].hex(), tx["input"][:10], tx["blockNumber"], new_level, tx["status"]
                ))
                if tx['from'] not in address_df:
                    adlist.append("{},{},{}\n".format(
                        tx["from"], tx["blockNumber"], new_level
                    ))
        except:
            return False

        with open(data[chain]['edgefile'], "a", encoding="UTF8") as tx_file:
                for string in txlist:
                    tx_file.write(string)
        with open(data[chain]['nodefile'], "a", encoding="UTF8") as ad_file:
            for string in adlist:
                ad_file.write(string)

        tmp_df = curr_level_address_df.copy()
        tmp_df.drop(index)
        tmp_df.to_csv('data/tocompute.csv', index=False)
    return True

In [61]:
exit = multi()
while exit == False:
    multi()

(2, 3)
(134, 3)
(240, 3)


KeyError: 'status'

In [ ]:

import requests

url = "https://eth-mainnet.g.alchemy.com/v2/demo"

payload = {
    "id": 1,
    "jsonrpc": "2.0",
    "method": "alchemy_getAssetTransfers",
    "params": [
        {
            "fromBlock": "0x0",
            "toBlock": "latest",
            "category": ["external"],
            "withMetadata": False,
            "excludeZeroValue": True,
            "maxCount": "0x3e8",
            "toAddress": "0x3e49460D0e95E645CFe61Fa36fe7C51FbEf40f41"
        }
    ]
}
headers = {
    "accept": "application/json",
    "content-type": "application/json"
}



response = requests.post(url, json=payload, headers=headers)
while response.status_code != 200:
    response = requests.post(url, json=payload, headers=headers)
history_txs = response.json()['result']['transfers']
for tx in history_txs:
    tx = {**w3.eth.get_transaction(tx['hash']), **w3.eth.get_transaction_receipt(tx['hash'])}
    try:
        print(tx['status'])
    except:
        print(tx)

1
1
1


In [ ]:
len({1:1})

1